In [1]:
from flask import Flask,request,render_template
import pandas as pd
import json
import tensorflow.keras as keras
import os
import plotly
import plotly.express as px
from Genre_Prediction import CNN
from Music_Recognizer import Music_Recognizer
from Spotify import Spotify
from Recommender import Recommender

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
def recommend(file_path=None,artist=None,title=None,required_features=['Danceability','Energy','Liveness','Tempo','Valence']):    
    #Creating objects for the respective classes
    c=CNN()
    m=Music_Recognizer()
    s=Spotify(SECRETS_PATH)
    r=Recommender(DATABASE_PATH)
   
    if(file_path):
        genre_dict=c.predict_genre(file_path,model)
        if(genre_dict==None):
            return render_template('Error_Unable_Extract_Features.html', filename=f.filename)

        (artist,title)=m.recognize_song_from_mp3(file_path,"best")   
        track_id=s.get_track_id(artist,title)
    else:
        track_id=s.get_track_id(artist,title)
        genre_list=s.get_genre_list(track_id)
        freq = lambda x: genre_list.count(x)
        percent = lambda z: freq(z)/len(genre_list)
        genre_prob_list=list(map(percent,set(genre_list)))
        genre_dict=dict()
        for index,genre in enumerate(set(genre_list)):
            genre_dict[genre]=genre_prob_list[index]

        l=['pop','hip hop','country','classical','jazz','blues','disco','reggae','rock','metal']
        d=[x for x in l if x not in list(genre_dict.keys())]
        index=0
        while(len(genre_dict)<10):
            genre_dict[d[index]]=0 
            index+=1
    print(genre_dict)
    #genre=genre_probability_dict['Genre'][genre_probability_dict['Probability'].index(max(genre_probability_dict['Probability']))]
    genre=max(zip(genre_dict.values(), genre_dict.keys()))[1]
    df=pd.DataFrame(list(zip(genre_dict.keys(),genre_dict.values())),columns= ['Genre',"Probability"])
    print(f"We recognized the genre to be : {genre}")  
    if(track_id!=None):
        present=r.check_id(track_id,None)
    else:
        present=False
    #Getting features using spotify API
    if(present==False):
        features=s.get_features(track_id,required_features)
        #pass object of spotify onto recommendations
        #track_id parameter will be false
        recommendations=r.recommend(features,genre,s,present,num_of_recommendations,measure)
    #Getting features from database
    else:
        features=r.get_features(track_id,required_features)
        #pass object of spotify onto recommendations
        #passing track_id 
        recommendations=r.recommend(features,genre,s,track_id,num_of_recommendations,measure)
    
    color_discrete_sequence = ['#ec7c34']*len(df)
    color_discrete_sequence[5] = '#609cd4'
    fig = px.bar(df, x='Genre', y='Probability', color='Probability', color_discrete_sequence=color_discrete_sequence,barmode='group')
    graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder) 
    return genre,recommendations,graphJSON

In [4]:
app = Flask(__name__)
    
@app.route('/')
def search_upload():
    return render_template("LP_search_or_upload.html")

@app.route("/search")
def search():
    return render_template("Input_search.html")

@app.route("/upload")
def upload():
    return render_template("Input_upload.html")
    
@app.route('/upload-recommendation',methods=['POST'])
def upload_recommendation():
    f = request.files['file'] 
    if(f.filename==""):
        return render_template('Error_Not_Required_Audio.html', filename=f.filename)
    required_features=request.form.getlist('Features')
    print(required_features)
    #samples,sample_rate=librosa.load()
    file_path=FLASK_SAVE_DIRECTORY+f.filename
    f.save(file_path) 
    if(os.path.splitext(file_path)[1].lower() not in ['.mp3','.wav']):
        return render_template('Error_Not_Required_Audio.html', filename=f.filename)
    genre,recommendations,graphJSON=recommend(file_path,None,None,required_features)
    return render_template('Result_recommendation.html', filename=f.filename,recommendations=recommendations,genre=genre, graphJSON=graphJSON)

@app.route('/search-recommendation',methods=['POST'])
def search_recommendation():
    artist = request.form.get('artist')
    title= request.form.get('title')
    song=artist+" "+title
    required_features=request.form.getlist('Features')
    print(required_features) 
    genre,recommendations,graphJSON=recommend(None,artist,title,required_features)
    return render_template('Result_recommendation.html', filename=song,recommendations=recommendations,genre=genre, graphJSON=graphJSON)

In [ ]:
if __name__ == "__main__":
    FLASK_SAVE_DIRECTORY=""
    MODEL_PATH=""
    SECRETS_PATH=""
    DATABASE_PATH=""
    genre_list=['Blues','Classical','Country','Disco','Hip-Hop','Jazz','Metal','Pop','Reggae','Rock']   
    SAMPLE_RATE = 22050
    TRACK_DURATION = 30
    num_mfcc=13
    n_fft=2048
    hop_length=512
    num_segments=10
    search_full=True
    num_of_recommendations=10
    measure="euclidean"
    model = keras.models.load_model(MODEL_PATH)
    app.run()